# Motor 
## Motor current of SGMADA 27063800
* Door weight : 336g
* Vcc= 6V
* Motor overload current : > 50mA
* Motor current during lifting : 37-43mA
* Motor current during no load : 19.5 - 21mA
* Motor current during lowering : 13.5 - 15mA

## Motor driver
Motor driver will be directly fed from the battery.  The back-EMF will be absorbed by the battery.

### TI DRV8231ADDAR
#### Current sense
* Select Rsense so that 30mA is in the middle of the ADC range.
* Rsense = 15Kohm
* Provide a 10nF capacitor parallel to Rsense (prevents premature trigger of current regulation)
* Vref = 2.25V 
  * input current to Vref not defined
  * Problem : power consumption of Vref when in sleep mode:
  * Solution : diode OR-ing of IN1 and IN2 charges a capacitor to Vx.  A TLV431, powered from Vx then provides Vref.

In [1]:
Imotor = 30e-3 # A
Vsense = 0.65 # V
Ai = 1500e-6 # A/A
Rsense = Vsense / (Imotor * Ai)

Rsense = 15e3
Itrip = 0.1 # A
Vref = Itrip * Ai * Rsense # V, must be between 0 and 3.6V

#### Low power sleep
* Input pins have 100Kohm pull-down
* Bringing  both inputs low puts the device in sleep mode
* Low power sleep current : 1µA (Ivmq)

#### Current regulation during startup and stall
* UVLO = 4.5V
* Add 47µF || 100nF on Vm
* Ramping up PWM might be a way to avoid tripping current regulation during startup.

### Toshiba TB67H450AFNG,EL
* Motor current flows through Rsense.

In [18]:
Itrip = 0.1
Rsense = 1
Ai = 10
Vref = Itrip * Ai * Rsense

When Rsense is 1ohm, MCU ADC attenuation should be ADC_ATTEN_DB_2_5 (max 1.05V input)

# MCU
ESP32-C3
* only 13 GPIO, but that should be enough for this application
* uses only half the current of ESP32-S3 in similar conditions (it's only single core)
* smaller than ESP32-S3 module
* 40% cheaper than ESP32-S3

## Programming & debug
* No source resistor needed on TXD, as the module already has R2, a 499(ohm?) resistor built in.
* Level-shifting circuitry to protect device from IO-programmer voltage, which might be 5V.
* ESP-Prog will be used for programming and debug.

## Motor 
### Motor pins
* 2 GPIOs needed.
* Pull-down in the motor controller, so there should be no pull-up on the MCU.  Reset state of the MCU pin can be found in [ESP32-C3 Technical reference manual](https://www.espressif.com/sites/default/files/documentation/esp32-c3_technical_reference_manual_en.pdf), §5.12 IO MUX Functions List.
* The glitches at power up are acceptable for this product.
* GPIO7 and GPIO10 will be used.

### Current sense connection
* Use ADC1, it's factory calibrated (ADC2 isn't)
* Lowest relative error in ATTEN2 range (up to 1.3V input) : [ADC_ATTEN_DB_6](https://docs.espressif.com/projects/arduino-esp32/en/latest/api/adc.html)
* Put a series resistor to limit the input current in case over-current events.
* GPIO3 is used.  Alternative pins is GPIO4.  GPIO2 is not suitable because of its required pull-up.

## RTC
* 32kHz crystal needed (is on ADC1.CH0 and ADC1.CH1).  RTC-counter can have other sources, but these are not accurate enough.
* Occupies GPIO0 & GPIO1

## Buttons
There are no three pins available for reading three buttons.  An extra requirement is that pushing a button should wake up the MCU in case it's sleeping.

This problem is solved by using a resistor divider circuit and connecting each button to it.  Capacitors are added to generate a falling edge (and wake up the MCU) at the start of the button press.

Only ADC-pins powered by the VDD3P3_RTC are suitable for this purpose :  GPIO2, GPIO3, GPIO4 (GPIO5 is connected to ADC2, to be avoided)

By using GPIO2, we can repurpose the mandatory bootstrapping pull-up.

## Display interface
I²C is used.  It requires only two GPIOs.  By using GPIO8 and GPIO9, we can repurpose the pull-up resistors which are already there for bootstrapping.

# Display
## IO expander

## I2C level translator

## Digital potentiometer

## Connectors
It should not be possible to reverse plug in the connectors.
* USB-C
* battery
* motor

# Web interface
## Button on webpage controls GPIO
Used technologies : websockets, javascript, html, css, JSON

* [ESP32 Remote Control with WebSocket](https://m1cr0lab-esp32.github.io/remote-control-with-websocket/)
* [ESP32 WebSocket Server: Control Outputs (Arduino IDE)](https://randomnerdtutorials.com/esp32-websocket-server-arduino/)

## Geo-location
[HTML Geolocation API](https://www.w3schools.com/html/html5_geolocation.asp)

## Time
[HTML element time](https://developer.mozilla.org/en-US/docs/Web/HTML/Element/input/time)

# Libraries
* [Solar Calculator](https://github.com/jpb10/SolarCalculator)
* [ArduinoJson](https://github.com/bblanchon/ArduinoJson)

### Visualize time
![sunset](https://icon-library.com/images/sunset-icon/sunset-icon-17.jpg)

* [Sunrise icon](https://icon-library.com/icon/sunrise-icon-png-16.html)

![Sunrise](https://icon-library.com/images/sunrise-icon-png/sunrise-icon-png-16.jpg)

* [Sunset icon](https://icon-library.com/icon/sunset-icon-17.html)
